In [1]:
import arrow
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import joblib 
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from sklearn.metrics import confusion_matrix

from models.SimpleAutoEncoder import SimpleAutoEncoder

from utils.evalUtils import calc_cm_metrics
from utils.evalUtils import print_confusion_matrix

In [2]:
from matplotlib import rc
rc('text', usetex=True)

In [3]:
%run -i ./scripts/setConfigs.py

Set configs..


In [4]:
os.getcwd()

'/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente'

In [5]:
%run -i ./scripts/EvalPreperations.py

Set configs..
Read the data..
Shape of normal data (X_sim): (105216, 17)
Shape of anormal data (X_test): (35040, 18)
Shape of drifted data (X_drifted): (35040, 18)
Shape of drifted anormal data (X_drifted,anormal): (35040, 19)
Save label..
Shape of anormal data (X_test): (35040, 17)
Shape of drifted data (X_drifted): (35040, 17)
Shape of drifted anormal data (X_drifted,anormal): (35040, 17)
Scale data..
Prepare data for PyTorch..


In [6]:
type(drifted_anormal_torch_tensor)

torch.Tensor

In [7]:
path = os.path.join(exp_data_path, 'experiment', 'fine_tuning', '*.csv')
result = glob.glob(path)

In [8]:
real_results = []
for r in result:
    if 'tVPII_M2' not in r:
        real_results.append(r)

In [9]:
real_results[0]

'/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_data/experiment/fine_tuning/20200410_EXPERIMENT_FINE_TUNED_MODEL__K_20_OPTIMIZER_Adam_NUM_ITER_64_LR_0_001_FILTER_CDE_[2]_MODEL_3.csv'

In [10]:
df_tVP2_m1 = pd.read_csv(real_results[0], sep=';')

In [11]:
for file in real_results[1:]:
    df = pd.read_csv(file, sep=';')
    df_tVP2_m1 = df_tVP2_m1.append(df)

In [12]:
df_tVP2_m1.head()

,optimizer,fine_tune_classes,name_pretrained_model,k,fine_tune_iterations,lr,model_fn,pretrained_model_fn,logreg_fn,TP_x_test,TN_x_test,FP_x_test,FN_x_test,TP_x_drifted_ano,TN_x_drifted_ano,FP_x_drifted_ano,FN_x_drifted_ano,Accuracy_x_test,Precision_x_test,Specifity_x_test,Sensitivity_x_test,Accuracy_x_drifted_ano,Precision_x_drifted_ano,Specifity_x_drifted_ano,Sensitivity_x_drifted_ano
0,Adam,[2],3,20,64,0.001,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2413,31468,1075,84,2412,28402,4141,85,96.692352,69.180046,92.878014,96.635963,87.939498,36.807569,92.172389,96.595915
0,SGD,"[1, 2, 3]",6,10,64,0.010,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2487,15098,17445,10,2487,13630,18913,10,50.185502,12.477423,85.857265,99.599519,45.996005,11.621495,84.569089,99.599519
0,SGD,"[2, 3]",5,5,1,0.010,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2456,28811,3732,41,2466,25853,6690,31,89.232306,39.689722,92.145073,98.358030,80.819064,26.933159,91.292065,98.758510
0,Adam,[1],3,10,1,0.001,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2440,30601,1942,57,2441,27641,4902,56,94.295091,55.682337,92.615236,97.717261,85.850457,33.242544,91.885513,97.757309
0,Adam,"[1, 3]",6,5,128,0.001,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2484,13784,18759,13,2485,12328,20215,12,46.426941,11.693264,84.730760,99.479375,42.274543,10.947137,83.224195,99.519423


In [15]:
df_tVP2_m1.sort_values(by='Precision_x_drifted_ano', ascending=True).head(3).iloc[0]

optimizer                                                                  SGD
fine_tune_classes                                                    [1, 2, 3]
name_pretrained_model                                                        5
k                                                                            5
fine_tune_iterations                                                       128
lr                                                                         0.1
model_fn                     /home/torge/dev/masterthesis_code/02_Experimen...
pretrained_model_fn          /home/torge/dev/masterthesis_code/02_Experimen...
logreg_fn                    /home/torge/dev/masterthesis_code/02_Experimen...
TP_x_test                                                                 2497
TN_x_test                                                                    0
FP_x_test                                                                32543
FN_x_test                                           

In [16]:
model_fn = df_tVP2_m1.sort_values(by='Precision_x_drifted_ano', ascending=True).head(3).iloc[0]['model_fn']

In [17]:
model = SimpleAutoEncoder(num_inputs=17, val_lambda=42)
print('Load weights..')
model.load_state_dict(torch.load(model_fn))
model.eval()

Load weights..


SimpleAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=17, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=8, bias=True)
    (3): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=17, bias=True)
    (3): Tanh()
  )
)

In [18]:
losses = []
for val in anormal_torch_tensor:
    loss = model.calc_reconstruction_error(val)
    losses.append(loss.item())

s_losses_anormal = pd.Series(losses)

X = s_losses_anormal.to_numpy()
X = X.reshape(-1, 1)

In [19]:
len(X)

35040

In [20]:
y = [1 if x > 0 else 0 for x in s_labels]
y_drifted = [1 if x > 0 else 0 for x in s_ano_labels_drifted_ano]

In [21]:
len(y)

35040

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
clf_meta = LogisticRegression(random_state=42, fit_intercept=True, solver='liblinear', class_weight={1:2.0})
clf_meta.fit(X, y)

LogisticRegression(C=1.0, class_weight={1: 2.0}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
losses_drifted_anormal = []
for val in drifted_anormal_torch_tensor:
    loss = model.calc_reconstruction_error(val)
    losses_drifted_anormal.append(loss.item())

s_losses_drifted_anormal = pd.Series(losses_drifted_anormal)

X_drifted_ano = s_losses_drifted_anormal.to_numpy()
X_drifted_ano = X_drifted_ano.reshape(-1, 1)

In [25]:
preds_x_drifted_ano = []
for val in X_drifted_ano:
    val = val.reshape(1,-1)
    pred = clf_meta.predict(val)
    preds_x_drifted_ano.append(pred[0])

In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from mlxtend.plotting import plot_confusion_matrix
import joblib

In [29]:
cm = confusion_matrix(y_drifted, preds_x_drifted_ano)
tn, fp, fn, tp  = cm.ravel()
print(cm)

[[29196  3347]
 [  256  2241]]


In [28]:
accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(int(tp), int(tn), int(fp), int(fn))
print('Accuracy: {}'.format(accuracy))
print('Precision: {}'.format(precision))
print('Specifity: {}'.format(specifity))
print('Sensitivity: {}'.format(sensitivity))
print('F1-Score: {}'.format(f1_score))

Accuracy: 89.71746575339907
Precision: 40.10379384387956
Specifity: 92.87145720008498
Sensitivity: 89.7476972363246
F1-Score: 55.43599257878115
